In [1]:
from arcgishub import hub

In [3]:
myHub = hub.Hub("https://www.arcgis.com", 'mmajumdar_dcdev')

Enter password: ········


#### Fetching a single initiative

In [4]:
walkability_init = myHub.initiatives.get('07fad5be49da45bea02c8f5b0876be24')

In [5]:
walkability_init.item

<Item title:"Walkability Assessment " type:Hub Initiative owner:mmajumdar_dcdev>

In [6]:
walkability_init.itemId

'07fad5be49da45bea02c8f5b0876be24'

#### Searching for initiatives

In [7]:
myHub.initiatives.search(title='Vision Zero', tags='Hub')

[<Initiative title:"Vision Zero - Demo (can be deleted)" owner:mshofner_dcdev>,
 <Initiative title:"Vision Zero - DC [JPP]" owner:jprather_dcdev>,
 <Initiative title:"Vision Zero Survey" owner:aturner>,
 <Initiative title:"Vision Zero_bw" owner:BWolfe_dcdev>,
 <Initiative title:"Vision Zero Tags" owner:aturner>,
 <Initiative title:"Nullvisjonen (Vision Zero) - v180115" owner:ghudgins5>,
 <Initiative title:"Vision Zero ManushiM" owner:mmajumdar_dcdev>,
 <Initiative title:"Vision Zero Palm Springs" owner:aturner>,
 <Initiative title:"Vision Zero with Collision Lens App Florida" owner:mmajumdar_dcdev>,
 <Initiative title:"Vision Zero Marvin" owner:mperry_dcdev>,
 <Initiative title:"Vision Zero - Graham" owner:ghudgins5>,
 <Initiative title:"Vision Zero - courtney" owner:cclaessens>,
 <Initiative title:"Vision Zero Test KC" owner:kcarl_dcdev>,
 <Initiative title:"Vision Zero Patrick" owner:phammons_dcdev>,
 <Initiative title:"Vision Zero Rosslyn" owner:aturner>,
 <Initiative title:"Vision 

#### Adding initiatives

In [8]:
item_dict = {"type":"Hub Initiative", "typekeywords":"Open Data", "title":"Python add initiative", 'description': 'filling something in'}

In [9]:
i = myHub.initiatives.add(item_dict)

In [10]:
i.item

<Item title:"Python add initiative" type:Hub Initiative owner:mmajumdar_dcdev>

#### Updating this initiative

In [11]:
item_dict['title'] = 'Python update initiative'

In [12]:
i.update(initiative_properties=item_dict)

True

In [13]:
i.item

<Item title:"Python update initiative" type:Hub Initiative owner:mmajumdar_dcdev>

#### Deleting initiative

In [14]:
i.delete()

True